In [1]:
!pip install datasets peft evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [2]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import evaluate
import torch
import numpy as np
import random

2024-05-25 10:21:39.024605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 10:21:39.024707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 10:21:39.154840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_name_or_path = "roberta-large"
num_epochs = 3
lr = 3e-5
batch_size = 8

In [4]:
dataset = load_dataset('multi_nli')

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'promptID': 31193,
 'pairID': '31193n',
 'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'premise_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )',
 'premise_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'hypothesis_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )',
 'hypothesis_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))',
 'genre': 'government',
 'label': 1}

In [6]:
metric = evaluate.load('accuracy')

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path, padding_side="right")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [9]:
def sample_10_percent(dataset):
    total_size = len(dataset)
    sample_size = max(1, total_size // 10)
    indices = random.sample(range(total_size), sample_size)
    return dataset.select(indices)

sampled_train_dataset = sample_10_percent(dataset['train'])
sampled_validation_dataset = sample_10_percent(dataset['validation_matched'])
sampled_train_dataset.shape, sampled_validation_dataset.shape

((39270, 10), (981, 10))

In [10]:
def tokenize_function(examples):
    outputs = tokenizer(examples['premise'], examples['hypothesis'], truncation=True, max_length=None)
    return outputs

In [11]:
train_tokenized_dataset = sampled_train_dataset.map(tokenize_function, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre"])
train_tokenized_dataset = train_tokenized_dataset.rename_column("label", "labels")

Map:   0%|          | 0/39270 [00:00<?, ? examples/s]

In [12]:
match_val_tokenized_dataset = sampled_validation_dataset.map(tokenize_function, batched=True, remove_columns=["promptID", "premise", "pairID", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre"])
match_val_tokenized_dataset = match_val_tokenized_dataset.rename_column("label", "labels")

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

In [13]:
train_tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 39270
})

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [15]:
target_modules = [
    "query",
    "keys",
    "values",
    "dense"
]
peft_config = LoraConfig(
    task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=0.5, lora_dropout=0.1, bias="none", target_modules=target_modules
)

In [16]:
model = RobertaForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, num_labels=3)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,821,571 || all params: 359,200,774 || trainable%: 1.0639


In [17]:
training_args = TrainingArguments(
    output_dir="pouya/roberta-large-peft-lora",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none",
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=match_val_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.770600,0.606669,0.752294
2,0.553000,0.455079,0.828746
3,0.516800,0.438224,0.839959


TrainOutput(global_step=14727, training_loss=0.6978298950609756, metrics={'train_runtime': 2874.538, 'train_samples_per_second': 40.984, 'train_steps_per_second': 5.123, 'total_flos': 1.5122864070586176e+16, 'train_loss': 0.6978298950609756, 'epoch': 3.0})